Importuję biblioteki

In [10]:

from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


Wczytuje artykuły z plików PDF

In [11]:
loader=DirectoryLoader('DANEPDF', glob="**/*.pdf", loader_cls=PyPDFLoader)
documents=loader.load()
print(f"Załadowano {len(documents)} dokumentów")

Załadowano 119 dokumentów


Dzielę artykuły na mniejsze fragmenty aby model lepiej działał, dodatkowo dla lepszego podziału dodaję separatory

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100, separators=["\n\n", "\n", ". ", " ", ""])
chunki = text_splitter.split_documents(documents)
print(f"Podzielono na {len(chunki)} fragmentów.")


Podzielono na 1841 fragmentów.


Wybieram 3 modele aby porównać z nich odpowiedzi oraz tworzę wektorową bazę danych dla każdego z nich, zapisuję ją na dysku aby przy kolejnych uruchomienach jedynie ją wczytywać

In [13]:
models={"jeden":"sentence-transformers/all-MiniLM-L6-v2", "dwa":"sentence-transformers/paraphrase-mpnet-base-v2", "trzy":"intfloat/e5-small"}
wektory={}
for n, model in models.items():
    e=HuggingFaceEmbeddings(model_name=model)
    w=Chroma.from_documents(documents=chunki, embedding=e, persist_directory=f"chroma_{n.lower()}", collection_name=f"fishing_{n.lower()}")
    w.persist()
    wektory[n]=w


Tworzę pipeline text-generation oparty na modelu facebookowym z tem 0.5 aby był trochę kreatywny ale bez szaleństwa, oraz wybieram procesor do wykonywania operacji ---> Update, nie chce mi się szczerze powiedziawszy szukać w jakiś bardziej zaawansowany sposób najlepszych parametrów do tego pipeline. Ale jako że generatorowi słów losowych jak go lubię nazywać, płyta się zacięła i zaczął gadać absolutne głupoty których pozwolę sobie tutaj nie przytoczyć, to po długiej eksploatacji motody prób i błędów dałem mu jeszcze no repeat ngram który ma zapobiegać powtarzaniu w kółko tych samych słów oraz early_stopping aby się zatrzymał jak znajdzie dobry punkt końcowy

PS: W pewnym momencie modele działając wbrew mojej woli zaczęły generować nieistniejące linki do yt lub zdjęć na dysku :P

In [26]:


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

qa_pipeline = pipeline(
    "text-generation",
    model="facebook/opt-125m",
    max_new_tokens=200,       
    do_sample=True,                
    temperature=0.5,         
    device=-1,
    no_repeat_ngram_size=2,
    early_stopping=True
)

qa_llm = HuggingFacePipeline(pipeline=qa_pipeline)
print("Zainicjalizowano model text-generation: facebook/opt-125m")


Zainicjalizowano model text-generation: facebook/opt-125m


Dla każdego z modelów tworzę "łańcuchy" które szukają podobieństw na k: 3 dokumantach, oraz z oddzielnymi pamięciami

In [27]:

qa_chains = {}

for nazwa_modelu, baza_wektorowa in wektory.items():
    try:
        retriever = baza_wektorowa.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}  
        )
        
        model_memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )
        
        chain = ConversationalRetrievalChain.from_llm(
            llm=qa_llm,
            retriever=retriever,
            memory=model_memory,
            return_source_documents=True,
            verbose=False
        )
        
        qa_chains[nazwa_modelu] = chain
        print(f"Utworzono chain dla modelu: {nazwa_modelu}")
        
    except Exception as e:
        print(f"Błąd dla modelu {nazwa_modelu}: {str(e)}")

Utworzono chain dla modelu: jeden
Utworzono chain dla modelu: dwa
Utworzono chain dla modelu: trzy


Pętla główna całego programu podczas której na zadane pytania odpowiadają wszystkie 3 modele a 3 ostatnie pytania są zapisywane do pamięci

In [28]:
print("Asystent QA uruchomiony!")
print("Wpisz 'exit' aby zakończyć.\n")

historia = []

while True:
    pytanie = input("Pytanie: ")
    
    if pytanie.lower() == "exit":
        break
    
    pierwsza_odpowiedz = None
    
    for nazwa, chain in qa_chains.items():
        print(f"\nMODEL: {nazwa}")
        print("-" * 25)
        
        try:
            wynik = chain({
                "question": pytanie,
                "chat_history": historia
            })
            
            if pierwsza_odpowiedz is None:
                pierwsza_odpowiedz = wynik.get("answer", "")
            
            odpowiedz = wynik.get("answer", "")
            dokumenty = wynik.get("source_documents", [])
            

            if odpowiedz and dokumenty:
                clean_answer = odpowiedz
                if "Use the following pieces of context" in clean_answer:
                    parts = clean_answer.split("Helpful Answer:")
                    if len(parts) > 1:
                        clean_answer = parts[1].strip()
                    else:
                        clean_answer = "Model nie wygenerował odpowiedzi"
                
                print(f"Odpowiedź: {clean_answer}")
                plik = dokumenty[0].metadata.get('source', '').split('\\')[-1]
                print(f"Źródło: {plik}")
            else:
                print("Brak odpowiedzi")
                
        except Exception as e:
            print(f"Błąd: {str(e)[:30]}...")
        
        print()
    
    if pierwsza_odpowiedz:
        historia.append((pytanie, pierwsza_odpowiedz[:100]))
        if len(historia) > 3:
            historia = historia[-3:]
    
    print("=" * 50)

Asystent QA uruchomiony!
Wpisz 'exit' aby zakończyć.


MODEL: jeden
-------------------------
Odpowiedź: The best time to fish in the USA is when the storm is expected to be low or very low. The best place to catch fish is in July. However, if the weather is not good, it is best to take a vacation. You can find out more about fishing in this article: https://www.discoverfisheries.com/article/fished-fishes-the-best-weather-for- Fishing-in-USA-July-20-2017-
However, in general, fishing is very expensive, and there are certain conditions that limit the amount of fishing you can do. Most of us will not fish for a full month, or even a year, because fishing does not allow you to get to your destination in time for the summer. But, there is a good chance you will find your way back to shore in a few months, which is exactly what you want. There are some situations where fishing can be very hard to do, but
Źródło: WP-2019-016.pdf


MODEL: dwa
-------------------------
Odpowiedź: We are planni